<a href="https://colab.research.google.com/github/synabreu/AIHUB/blob/master/seq2seq_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

New Section

In [0]:
batch_size = 128 # 64
epochs = 100 # 1000
latent_dim = 1024
num_samples = 10000

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# data_path = 'drive/My Drive/Colab Notebooks/kor-utf8_2.txt'  
data_path = 'drive/My Drive/Colab Notebooks/KOR_190920_UTF8.txt'  


In [0]:
# Data Loading Test
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

lines= pd.read_table(data_path, names=['eng', 'kor'])
lines = lines[0:10000]
lines.shape
lines.sample(10)

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [0]:
with open(data_path, encoding='UTF8') as f:
# with open(data_path, encoding='UTF16') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    # input_text, target_text = ast.literal_eval(line.split('\t'))
    input_text, target_text = line.split('\t')
    
    target_text = '\t' + target_text + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [0]:
input_characters = sorted(list(input_characters))
target_characters = set(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [0]:
print('Number of Samples:', len(input_texts))
print('Number of Unique Input Tokens:', num_encoder_tokens)
print('Number of Unique Output Tokens:', num_decoder_tokens)
print('Max Sequence Length for Inputs:', max_encoder_seq_length)
print('Max Sequence Length of Outputs:', max_decoder_seq_length)

In [0]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [0]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [0]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [0]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [0]:
save_path = 'drive/My Drive/Colab Notebooks/s2s_0504_01.h5'  
model.save(save_path)

model.summary()